In [7]:
import numpy as np
import nltk
#nltk.download('punkt_tab')
from transformers import AutoModelForCausalLM, AutoTokenizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer

In [8]:
# List of labels.   #everything must be in lower case.   # underscore needed for more than one word as input label

#labels = ["hypolimnas_misippus", "danaus_chrysippus", "amauris_ochlea", "acraea_egina"]
labels = ["heartwood", "sapwood"]

# Generate Contextual Paragraphs Using an LLM

In [9]:
# Load the Qwen2.5-1.5B-Instruct model and tokenizer        #most download
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).half().cuda()

In [ ]:

# Function to generate text for a label
def generate_paragraph(label):
    
    prompt = f"Write a factual and descriptive paragraph about {label}, focus on thier characteristics, description, differences and uses"
    
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=250, temperature=0.7)
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Generate paragraphs for each label
contextual_paragraphs_Qwen1_5B = {label: generate_paragraph(label) for label in labels}

# Print the generated paragraphs
for label, paragraph in contextual_paragraphs_Qwen1_5B.items():
    
    print(f"{label}: {paragraph}\n")

heartwood: Write a factual and descriptive paragraph about heartwood, focus on thier characteristics, description and anatomy. 250-300 words

Heartwood is the thick central core of a tree that contains no living cells. Unlike sapwood, which is made up of dead but active wood cells, heartwood is composed entirely of dead cells. This makes it more resistant to decay and pests than sapwood.

The heartwood in trees develops over time as they grow larger. As the innermost parts of the trunk or branches age and die, they are replaced by new growth from the cambium layer. The result is a strong, durable wood with beautiful grain patterns that vary widely depending on species.

Heartwood can be identified visually because its color is usually darker and often banded or mottled compared to surrounding sapwood. It may also have knots or irregularities due to past damage or stress during the tree's life cycle. In woodworking, artisans carefully select pieces of heartwood for their strength and ae

#  Prepare Corpus for Word Embedding

Combine the generated paragraphs into a single corpus

In [11]:
corpus = "\n".join(contextual_paragraphs_Qwen1_5B.values())

print(corpus)

Write a factual and descriptive paragraph about heartwood, focus on thier characteristics, description and anatomy. 250-300 words

Heartwood is the thick central core of a tree that contains no living cells. Unlike sapwood, which is made up of dead but active wood cells, heartwood is composed entirely of dead cells. This makes it more resistant to decay and pests than sapwood.

The heartwood in trees develops over time as they grow larger. As the innermost parts of the trunk or branches age and die, they are replaced by new growth from the cambium layer. The result is a strong, durable wood with beautiful grain patterns that vary widely depending on species.

Heartwood can be identified visually because its color is usually darker and often banded or mottled compared to surrounding sapwood. It may also have knots or irregularities due to past damage or stress during the tree's life cycle. In woodworking, artisans carefully select pieces of heartwood for their strength and aesthetic qua

# Generate Word Embeddings

Word2Vec Embeddings

In [21]:
# Tokenize the corpus into sentences
sentences = [word_tokenize(paragraph.lower()) for paragraph in contextual_paragraphs_Qwen1_5B.values()]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=300, window=5, min_count=1, workers=4)

# Retrieve embeddings for each label
label_embeddings = {label: word2vec_model.wv[label] for label in labels}

print(label_embeddings)

{'heartwood': array([-2.3906983e-03,  1.4879876e-03,  7.1068550e-04,  2.5278607e-03,
       -1.6401529e-03, -1.6169355e-03, -1.9412265e-03,  1.2758502e-03,
       -1.4936653e-03,  2.7830193e-03, -1.3968637e-03, -3.0820339e-03,
       -1.6715266e-03,  2.0953901e-03, -2.1831659e-03, -1.7706989e-03,
       -2.3743915e-03,  2.0804645e-03,  1.1151930e-03,  9.5779955e-04,
       -1.0721458e-03,  2.0030313e-03, -2.0210496e-03, -6.5585307e-04,
       -1.9203986e-03, -3.3742501e-04, -7.6623954e-04,  2.8584402e-03,
        1.7482278e-05, -2.9280968e-03, -1.7468358e-03, -2.3055056e-03,
        9.4320334e-04,  3.1781583e-03, -2.0049938e-03,  2.7731869e-03,
        2.9197580e-03, -2.4057268e-03, -2.9476017e-03,  3.1929407e-03,
        2.7370697e-03, -1.5731459e-03, -2.2511003e-03,  2.5530702e-03,
        1.3298662e-03,  2.7736013e-03, -2.4622390e-03, -3.1398213e-03,
        5.1787525e-04, -3.2352987e-03, -1.6192313e-03, -1.1772988e-03,
        3.1772223e-03,  2.8669876e-03, -9.4482274e-04,  2.03680

In [ ]:
# Get the vocabulary list
vocabulary_list = list(word2vec_model.wv.key_to_index.keys())

# Print the vocabulary list
print(vocabulary_list)

['.', ',', 'and', 'the', 'of', 'heartwood', 'a', 'sapwood', 'is', 'to', 'it', 'in', 'or', 'for', 'wood', 'its', 'cells', 'characteristics', 'woodworking', 'has', 'on', 'this', 'about', 'as', 'tree', 'that', 'than', 'they', 'making', 'decay', 'durability', 'write', 'by', 'important', 'from', 'overall', 'resistant', 'strength', 'with', 'can', 'due', 'be', 'also', 'compared', 'color', 'layer', 'often', 'more', 'focus', 'anatomy', 'description', 'dead', 'which', 'living', 'thier', 'makes', 'paragraph', 'descriptive', 'factual', 'contains', 'result', 'thick', 'strong', 'durable', 'words', 'beautiful', '250-300', 'widely', 'grain', 'patterns', 'vary', 'core', 'depending', 'species', 'identified', 'visually', 'because', 'usually', 'central', 'no', 'cambium', 'growth', 'entirely', 'pests', 'composed', 'active', 'darker', 'develops', 'over', 'time', 'but', 'grow', 'up', 'larger', 'innermost', 'parts', 'trunk', 'made', 'branches', 'age', 'die', 'are', 'replaced', 'unlike', 'new', 'trees', 'opera

BERT Embeddings

In [18]:
# Load a pre-trained BERT model
bert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each label's paragraph
label_embeddings_b = {label: bert_model.encode(paragraph) for label, paragraph in contextual_paragraphs_Qwen1_5B.items()}

print(label_embeddings_b)

{'heartwood': array([ 4.77939956e-02, -3.07247657e-02,  1.08484374e-02, -5.37702115e-04,
        7.12271333e-02, -1.95555110e-02, -1.82913672e-02, -2.57748812e-02,
        7.48094842e-02,  6.53318837e-02, -4.71443981e-02,  1.98347699e-02,
       -3.66703384e-02,  1.30697386e-03, -1.00413729e-02, -2.11795755e-02,
        1.64323058e-02, -3.54270674e-02, -4.55403030e-02,  3.29334587e-02,
        5.86275086e-02,  6.15587756e-02, -5.75726554e-02, -3.61469854e-03,
        2.81673372e-02, -4.30600047e-02, -8.33306760e-02, -1.27847819e-02,
        5.31014241e-03,  1.38385193e-02,  1.09778389e-01,  1.91196874e-02,
        6.22714907e-02,  7.61768222e-02, -8.57131407e-02,  6.80446327e-02,
        3.15507012e-03, -6.39113635e-02,  3.06881405e-02,  3.92321981e-02,
       -6.77363500e-02,  2.43599452e-02, -2.16351170e-02, -1.05603347e-02,
        4.03676229e-03,  1.19033894e-02, -1.18596349e-02, -5.47433943e-02,
       -4.55513299e-02, -1.57920141e-02, -6.02350980e-02, -7.51601160e-02,
       -5.1

In [22]:
# Aggregate embeddings by averaging
final_vector = np.mean(list(label_embeddings.values()), axis=0)
print("Final Word2Vec Vector Shape:", final_vector.shape)

Final Word2Vec Vector Shape: (300,)


In [20]:
# Aggregate embeddings by averaging
final_vector = np.mean(list(label_embeddings_b.values()), axis=0)
print("Final BERT Vector Shape:", final_vector.shape)

Final BERT Vector Shape: (384,)
